# Kaggle Credit Card Fraud Detection Using Neural Networks

## Dataset Description and Notebook Objectives

### Description

From the [Kaggle Dataset](https://www.kaggle.com/datasets/mlg-ulb/creditcardfraud)'s description: "The dataset contains transactions made by credit cards in September 2013 by European cardholders.
This dataset presents transactions that occurred in two days, where we have 492 frauds out of 284,807 transactions. The dataset is highly unbalanced, the positive class (frauds) account for 0.172% of all transactions. It contains only numerical input variables which are the result of a PCA transformation... due to confidentiality issues, we cannot provide the original features and more background information about the data. Features V1, V2, … V28 are the principal components obtained with PCA, the only features which have not been transformed with PCA are 'Time' and 'Amount'. Feature 'Time' contains the seconds elapsed between each transaction and the first transaction in the dataset. The feature 'Amount' is the transaction Amount, this feature can be used for example-dependant cost-sensitive learning. Feature 'Class' is the response variable and it takes value 1 in case of fraud and 0 otherwise."

The dataset is provided in csv format as creditcard.csv. The file can be found on Kaggle at the link provided above, and is not included in this GitHub repository.

### Objectives

Objective: Create a machine learning algorithm that can accurately detect fraudulent transactions.

Given that our dataset contains PCA components, rather than the features themselves, there is little reason (with respect to the end result) to track feature importance. Coupled with the fact that the analyses contained within this repository are for practicing different data science skills, this is an excellent opportunity to leverage the use of a neural network.

The Kaggle description recommends measuring accuracy with Area Under the Precision-Recall Curve (AUPRC) because "confusion matrix accuracy is not meaningful for unbalanced classification." Perhaps this is true when using weighted recall (the default CM accuracy metric) but a more preferable metric in this instance, with respect to the confusion matrix, is to use an unweighted F1-score. For learning purposes, we will explore both Area-Under metrics, as well as this unweighted F1-score.

In [1]:
# Import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

from sklearn.metrics import f1_score

## First Iteration

### EDA

Let's take a quick look at our features and see what they look like.

In [2]:
df = pd.read_csv('creditcard.csv')

In [3]:
df.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 284807 entries, 0 to 284806
Data columns (total 31 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   Time    284807 non-null  float64
 1   V1      284807 non-null  float64
 2   V2      284807 non-null  float64
 3   V3      284807 non-null  float64
 4   V4      284807 non-null  float64
 5   V5      284807 non-null  float64
 6   V6      284807 non-null  float64
 7   V7      284807 non-null  float64
 8   V8      284807 non-null  float64
 9   V9      284807 non-null  float64
 10  V10     284807 non-null  float64
 11  V11     284807 non-null  float64
 12  V12     284807 non-null  float64
 13  V13     284807 non-null  float64
 14  V14     284807 non-null  float64
 15  V15     284807 non-null  float64
 16  V16     284807 non-null  float64
 17  V17     284807 non-null  float64
 18  V18     284807 non-null  float64
 19  V19     284807 non-null  float64
 20  V20     284807 non-null  float64
 21  V21     28

In [5]:
df.describe()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
count,284807.000000,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,...,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,284807.000000,284807.000000
mean,94813.859575,1.168375e-15,3.416908e-16,-1.379537e-15,2.074095e-15,9.604066e-16,1.487313e-15,-5.556467e-16,1.213481e-16,-2.406331e-15,...,1.654067e-16,-3.568593e-16,2.578648e-16,4.473266e-15,5.340915e-16,1.683437e-15,-3.660091e-16,-1.227390e-16,88.349619,0.001727
std,47488.145955,1.958696e+00,1.651309e+00,1.516255e+00,1.415869e+00,1.380247e+00,1.332271e+00,1.237094e+00,1.194353e+00,1.098632e+00,...,7.345240e-01,7.257016e-01,6.244603e-01,6.056471e-01,5.212781e-01,4.822270e-01,4.036325e-01,3.300833e-01,250.120109,0.041527
min,0.000000,-5.640751e+01,-7.271573e+01,-4.832559e+01,-5.683171e+00,-1.137433e+02,-2.616051e+01,-4.355724e+01,-7.321672e+01,-1.343407e+01,...,-3.483038e+01,-1.093314e+01,-4.480774e+01,-2.836627e+00,-1.029540e+01,-2.604551e+00,-2.256568e+01,-1.543008e+01,0.000000,0.000000
25%,54201.500000,-9.203734e-01,-5.985499e-01,-8.903648e-01,-8.486401e-01,-6.915971e-01,-7.682956e-01,-5.540759e-01,-2.086297e-01,-6.430976e-01,...,-2.283949e-01,-5.423504e-01,-1.618463e-01,-3.545861e-01,-3.171451e-01,-3.269839e-01,-7.083953e-02,-5.295979e-02,5.600000,0.000000
50%,84692.000000,1.810880e-02,6.548556e-02,1.798463e-01,-1.984653e-02,-5.433583e-02,-2.741871e-01,4.010308e-02,2.235804e-02,-5.142873e-02,...,-2.945017e-02,6.781943e-03,-1.119293e-02,4.097606e-02,1.659350e-02,-5.213911e-02,1.342146e-03,1.124383e-02,22.000000,0.000000
75%,139320.500000,1.315642e+00,8.037239e-01,1.027196e+00,7.433413e-01,6.119264e-01,3.985649e-01,5.704361e-01,3.273459e-01,5.971390e-01,...,1.863772e-01,5.285536e-01,1.476421e-01,4.395266e-01,3.507156e-01,2.409522e-01,9.104512e-02,7.827995e-02,77.165000,0.000000
max,172792.000000,2.454930e+00,2.205773e+01,9.382558e+00,1.687534e+01,3.480167e+01,7.330163e+01,1.205895e+02,2.000721e+01,1.559499e+01,...,2.720284e+01,1.050309e+01,2.252841e+01,4.584549e+00,7.519589e+00,3.517346e+00,3.161220e+01,3.384781e+01,25691.160000,1.000000


Many of the values are hard to parse when presented in scientific format. Let's change that.

In [6]:
pd.options.display.float_format = '{:.6f}'.format

df.describe()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
count,284807.000000,284807.000000,284807.000000,284807.000000,284807.000000,284807.000000,284807.000000,284807.000000,284807.000000,284807.000000,...,284807.000000,284807.000000,284807.000000,284807.000000,284807.000000,284807.000000,284807.000000,284807.000000,284807.000000,284807.000000
mean,94813.859575,0.000000,0.000000,-0.000000,0.000000,0.000000,0.000000,-0.000000,0.000000,-0.000000,...,0.000000,-0.000000,0.000000,0.000000,0.000000,0.000000,-0.000000,-0.000000,88.349619,0.001727
std,47488.145955,1.958696,1.651309,1.516255,1.415869,1.380247,1.332271,1.237094,1.194353,1.098632,...,0.734524,0.725702,0.624460,0.605647,0.521278,0.482227,0.403632,0.330083,250.120109,0.041527
min,0.000000,-56.407510,-72.715728,-48.325589,-5.683171,-113.743307,-26.160506,-43.557242,-73.216718,-13.434066,...,-34.830382,-10.933144,-44.807735,-2.836627,-10.295397,-2.604551,-22.565679,-15.430084,0.000000,0.000000
25%,54201.500000,-0.920373,-0.598550,-0.890365,-0.848640,-0.691597,-0.768296,-0.554076,-0.208630,-0.643098,...,-0.228395,-0.542350,-0.161846,-0.354586,-0.317145,-0.326984,-0.070840,-0.052960,5.600000,0.000000
50%,84692.000000,0.018109,0.065486,0.179846,-0.019847,-0.054336,-0.274187,0.040103,0.022358,-0.051429,...,-0.029450,0.006782,-0.011193,0.040976,0.016594,-0.052139,0.001342,0.011244,22.000000,0.000000
75%,139320.500000,1.315642,0.803724,1.027196,0.743341,0.611926,0.398565,0.570436,0.327346,0.597139,...,0.186377,0.528554,0.147642,0.439527,0.350716,0.240952,0.091045,0.078280,77.165000,0.000000
max,172792.000000,2.454930,22.057729,9.382558,16.875344,34.801666,73.301626,120.589494,20.007208,15.594995,...,27.202839,10.503090,22.528412,4.584549,7.519589,3.517346,31.612198,33.847808,25691.160000,1.000000


There are no explicit nulls in the data, but having a purchase amount of $0.00 seems odd. Let's take a look at those.

In [7]:
df.loc[df['Amount']==0]

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
383,282.000000,-0.356466,0.725418,1.971749,0.831343,0.369681,-0.107776,0.751610,-0.120166,-0.420675,...,0.020804,0.424312,-0.015989,0.466754,-0.809962,0.657334,-0.043150,-0.046401,0.000000,0
514,380.000000,-1.299837,0.881817,1.452842,-1.293698,-0.025105,-1.170103,0.861610,-0.193934,0.592001,...,-0.272563,-0.360853,0.223911,0.598930,-0.397705,0.637141,0.234872,0.021379,0.000000,0
534,403.000000,1.237413,0.512365,0.687746,1.693872,-0.236323,-0.650232,0.118066,-0.230545,-0.808523,...,-0.077543,-0.178220,0.038722,0.471218,0.289249,0.871803,-0.066884,0.012986,0.000000,0
541,406.000000,-2.312227,1.951992,-1.609851,3.997906,-0.522188,-1.426545,-2.537387,1.391657,-2.770089,...,0.517232,-0.035049,-0.465211,0.320198,0.044519,0.177840,0.261145,-0.143276,0.000000,1
575,430.000000,-1.860258,-0.629859,0.966570,0.844632,0.759983,-1.481173,-0.509681,0.540722,-0.733623,...,0.268028,0.125515,-0.225029,0.586664,-0.031598,0.570168,-0.043007,-0.223739,0.000000,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
283719,171817.000000,-0.750414,0.904175,0.996461,0.427284,1.720336,0.929256,0.794272,0.176719,-1.836261,...,0.050750,0.115532,-0.623995,-0.186896,0.733759,2.558151,-0.188835,0.001654,0.000000,0
283782,171870.000000,2.083677,-0.065811,-1.442870,0.135416,0.043035,-1.306975,0.335835,-0.371635,0.730560,...,-0.147536,-0.246599,0.194758,-0.082277,0.012887,-0.069278,-0.048995,-0.065482,0.000000,0
283949,172027.000000,2.132569,-0.057836,-1.724522,-0.030326,0.412146,-0.903088,0.345843,-0.348132,0.722638,...,-0.188739,-0.343876,0.105024,-0.763831,0.117381,-0.027682,-0.047514,-0.071700,0.000000,0
284085,172140.000000,-2.210521,-1.039425,0.189704,-1.291932,3.742120,-1.665061,3.120388,-2.324089,0.364926,...,-0.286359,1.326003,-0.361764,-0.268117,1.051309,0.334629,-1.930149,-0.899888,0.000000,0


In [8]:
df.loc[df['Amount']==0]['Class'].value_counts()

0    1798
1      27
Name: Class, dtype: int64

Not good. It's tempting to drop these observations, as a $0 purchase seems like bad data, but when there's quite a few frauds in this set, it's better to leave it in for now.

### No Need For Time

The 'Time' column doesn't accomplish much - given actual features (instead of PCA transformations) we may be able to compare times for individual accounts and find purchases in quick succession that don't make much sense, but because the times are only in relation to the start of the dataset (and not to each other in any way) we can drop it.

In [9]:
df = df.drop(columns=['Time'])

### The Impact of Outliers

Per [this study](https://scialert.net/fulltext/?doi=jas.2005.1394.1398#:~:text=The%20neural%20network%20is%20unaffected,tests%20as%20the%20training%20data.), with respect to neural networks, "when the percentage-outliers is lower than 15% (even though the magnitude of outliers may increase), the effect on model accuracy is statistically insignificant." Let's take a look at our data and see if there are any features with a percentage of outliers greater than 15%.

In [10]:
# calculates feature limits
def get_feature_limits(series):
    Q1 = series.quantile(0.25)
    Q3 = series.quantile(0.75)
    IQR = Q3 - Q1
    lower_lim = Q1 - 1.5*IQR
    upper_lim = Q3 + 1.5*IQR
    return lower_lim, upper_lim

In [11]:
# percent of outliers by feature
for feature in df.columns[:29]:
    lower, upper = get_feature_limits(df[feature])
    print(feature, ((df[feature] < lower) | (df[feature] > upper)).value_counts(normalize=True)[1]*100)

V1 2.4795738868777804
V2 4.7491810243428
V3 1.1807996292225964
V4 3.9142296362097846
V5 4.316958501722219
V6 8.063355184388024
V7 3.1417767119487934
V8 8.47380857914307
V9 2.9082852598426303
V10 3.334187713082895
V11 0.27386967314707855
V12 5.3889124916171305
V13 1.182555204050462
V14 4.96792564789489
V15 1.0161267103687759
V16 2.873524878250886
V17 2.605273044552978
V18 2.644949035662747
V19 3.5831282236742776
V20 9.750462593967143
V21 5.090113655914355
V22 0.4624184096598749
V23 6.510022576692287
V24 1.67622284564635
V25 1.8844340202312442
V26 1.9648393473475019
V27 13.750715396742356
V28 10.653530285421356
Amount 11.201971861646658


None of our features have >15% outliers. V27 gets close with 13.75%, but no more. This is good news! We don't have to trim our dataset of outliers before we begin modeling.

### Metrics and Baseline

As stated above, we will use an unweighted F1-score to determine accuracy. Our baseline prediction will be all observations classified as 0 (not fraud).

In [12]:
X = df.drop(columns=['Class'])
y = df['Class']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

In [13]:
base_pred = np.zeros(X_test.shape[0])

In [14]:
print(f1_score(y_test, base_pred))

0.0


### Modeling

A solid basic neural network model has the following structure.

1. An input layer with nodes.count = features.count
2. A hidden layer with features.count / 2
3. An output layer = target.unique_values.count

In [16]:
model = Sequential()

model.add(Dense(
    15,
    input_dim=29,
    activation='sigmoid'
))

model.add(Dense(1, activation='sigmoid'))

model.compile(
    loss='binary_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)

model.fit(X_train, y_train, epochs=100)

Epoch 1/100
7121/7121 [==============================] - 6s 771us/step - loss: 0.0147 - accuracy: 0.9979
Epoch 2/100
7121/7121 [==============================] - 5s 763us/step - loss: 0.0037 - accuracy: 0.9992
Epoch 3/100
7121/7121 [==============================] - 5s 764us/step - loss: 0.0033 - accuracy: 0.9993
Epoch 4/100
7121/7121 [==============================] - 5s 752us/step - loss: 0.0032 - accuracy: 0.9994
Epoch 5/100
7121/7121 [==============================] - 5s 745us/step - loss: 0.0032 - accuracy: 0.9993
Epoch 6/100
7121/7121 [==============================] - 5s 767us/step - loss: 0.0032 - accuracy: 0.9994
Epoch 7/100
7121/7121 [==============================] - 5s 755us/step - loss: 0.0031 - accuracy: 0.9994
Epoch 8/100
7121/7121 [==============================] - 5s 756us/step - loss: 0.0030 - accuracy: 0.9994
Epoch 9/100
7121/7121 [==============================] - 5s 757us/step - loss: 0.0031 - accuracy: 0.9994
Epoch 10/100
7121/7121 [==============================]

In [17]:
train_scores = model.evaluate(X_train, y_train)
test_scores = model.evaluate(X_test, y_test)
print('Train Score')
print(f'{model.metrics_names[1]}: {train_scores[1]*100}')
print('Test Score')
print(f'{model.metrics_names[1]}: {test_scores[1]*100}')

1781/1781 [==============================] - 1s 696us/step - loss: 0.0032 - accuracy: 0.9992
Train Score
accuracy: 99.95216131210327
Test Score
accuracy: 99.92451071739197


In [20]:
predictions = model.predict(X_test)

1781/1781 [==============================] - 1s 615us/step


In [23]:
pred_classes = np.around(predictions)

In [26]:
print(f1_score(y_test, pred_classes))

0.7817258883248731
